[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/03_Complete_Visualization_Suite.ipynb)

# Complete Visualization Suite

## Overview

Comprehensive visualization capabilities: visualize knowledge graphs, embeddings, quality metrics, analytics, and temporal data.

**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/visualization/)

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```


In [ ]:
%pip install -U "semantica[all]"
import semantica
print(semantica.__version__)


In [ ]:
from semantica.visualization import (
    KGVisualizer,
    EmbeddingVisualizer,
    QualityVisualizer,
    AnalyticsVisualizer,
    TemporalVisualizer
)
from semantica.kg import GraphBuilder, GraphAnalyzer
from semantica.embeddings import EmbeddingGenerator
import numpy as np


## Step 1: Create Sample Knowledge Graph


In [ ]:
builder = GraphBuilder()

entities = [
    {"id": "e1", "type": "Person", "name": "Alice", "properties": {"age": 30}},
    {"id": "e2", "type": "Person", "name": "Bob", "properties": {"age": 35}},
    {"id": "e3", "type": "Organization", "name": "Tech Corp", "properties": {"founded": 2010}},
    {"id": "e4", "type": "Location", "name": "San Francisco", "properties": {"country": "USA"}},
]

relationships = [
    {"source": "e1", "target": "e2", "type": "knows", "properties": {"since": 2020}},
    {"source": "e1", "target": "e3", "type": "works_for", "properties": {"role": "Engineer"}},
    {"source": "e3", "target": "e4", "type": "located_in", "properties": {}},
]

knowledge_graph = builder.build(entities, relationships)


## Step 2: Knowledge Graph Visualization


In [ ]:
kg_visualizer = KGVisualizer(layout="force", color_scheme="vibrant")
kg_visualizer.visualize_network(knowledge_graph, output="interactive")


## Step 3: Generate Embeddings and Visualize


In [ ]:
embedding_generator = EmbeddingGenerator()
texts = [entity.get("name", "") for entity in entities]
embeddings = embedding_generator.generate_embeddings(texts, data_type="text")

labels = [entity.get("type", "Unknown") for entity in entities]

embedding_visualizer = EmbeddingVisualizer()
embedding_visualizer.visualize_2d_projection(embeddings, labels, method="tsne", output="interactive", file_path=None)


## Step 4: Quality Metrics Visualization


In [ ]:
quality_visualizer = QualityVisualizer()
quality_report = {
    "overall_score": 0.85,
    "consistency_score": 0.90,
    "completeness_score": 0.80
}
quality_visualizer.visualize_dashboard(quality_report, output="interactive")


## Step 5: Graph Analytics Visualization


In [ ]:
graph_analyzer = GraphAnalyzer()

centrality_results = graph_analyzer.calculate_centrality(
    knowledge_graph, 
    centrality_type="degree"
)

centrality_scores = {}
if centrality_results and "centrality_measures" in centrality_results:
    degree_centrality = centrality_results["centrality_measures"].get("degree", {})
    if isinstance(degree_centrality, dict) and "centrality" in degree_centrality:
        centrality_scores = degree_centrality["centrality"]
    elif isinstance(degree_centrality, dict):
        centrality_scores = degree_centrality

communities_result = graph_analyzer.detect_communities(
    knowledge_graph, 
    algorithm="louvain"
)

communities = []
community_dict = {}
if communities_result and "communities" in communities_result:
    communities_data = communities_result["communities"]
    if isinstance(communities_data, list):
        communities = communities_data
        for idx, community in enumerate(communities):
            if isinstance(community, list):
                for node in community:
                    community_dict[node] = idx
            elif isinstance(community, dict) and "nodes" in community:
                for node in community["nodes"]:
                    community_dict[node] = idx

analytics_visualizer = AnalyticsVisualizer()
analytics_visualizer.visualize_centrality(centrality_scores, title="Node Centrality Scores")

if community_dict:
    analytics_visualizer.visualize_communities(
        knowledge_graph, 
        community_dict, 
        title="Community Detection"
    )


## Step 6: Temporal Data Visualization


In [ ]:
temporal_kg = {
    "entities": entities,
    "relationships": relationships,
    "timestamps": {
        "e1": [2020, 2021, 2022],
        "e2": [2020, 2021],
        "e3": [2010, 2015, 2020, 2022],
    }
}

entity_history = {
    "e1": [
        {"timestamp": 2020, "properties": {"age": 28}},
        {"timestamp": 2021, "properties": {"age": 29}},
        {"timestamp": 2022, "properties": {"age": 30}},
    ]
}

from semantica.kg import TemporalVersionManager
temporal_visualizer = TemporalVisualizer()
temporal_visualizer.visualize_timeline(temporal_kg, output="interactive")
# Convert entity history to metrics for visualization
timestamps = [str(item["timestamp"]) for item in entity_history["e1"]]
age_values = [item["properties"]["age"] for item in entity_history["e1"]]
metrics_history = {"age": age_values}
temporal_visualizer.visualize_metrics_evolution(metrics_history, timestamps, output="interactive")

# Create versions for snapshot comparison
version_manager = TemporalVersionManager()
v1 = version_manager.create_version(temporal_kg, timestamp="2020-01-01", version_label="v2020")
temporal_kg_v2 = {
    "entities": temporal_kg.get("entities", []),
    "relationships": temporal_kg.get("relationships", []) + [
        {"source": "e1", "target": "e2", "type": "collaborated_with", "valid_from": "2023-01-01"}
    ]
}
v2 = version_manager.create_version(temporal_kg_v2, timestamp="2023-01-01", version_label="v2023")
snapshots = {v1["timestamp"]: v1, v2["timestamp"]: v2}
temporal_visualizer.visualize_snapshot_comparison(snapshots, output="interactive")
version_history = [
    {"version": v1.get("label"), "timestamp": v1.get("timestamp")},
    {"version": v2.get("label"), "timestamp": v2.get("timestamp")}
]
temporal_visualizer.visualize_version_history(version_history, output="interactive")


## Summary

All visualization types demonstrated:
- Knowledge Graph Visualization
- Embedding Visualization (t-SNE)
- Quality Metrics Visualization
- Graph Analytics Visualization (Centrality & Communities)
- Temporal Data Visualization (Timeline & Evolution)
